In [6]:
import os
import openai
import json
import pandas as pd
from datasets import load_dataset

os.environ["OPENAI_API_KEY"] = ''
from openai import OpenAI
client = OpenAI()

languages = [
    "acm_Arab", "afr_Latn", "als_Latn", "amh_Ethi", "apc_Arab", "arb_Arab", "arb_Latn", "ars_Arab",
    "ary_Arab", "arz_Arab", "asm_Beng", "azj_Latn", "bam_Latn", "ben_Beng", "ben_Latn", "bod_Tibt",
    "bul_Cyrl", "cat_Latn", "ceb_Latn", "ces_Latn", "ckb_Arab", "dan_Latn", "deu_Latn", "ell_Grek",
    "eng_Latn", "est_Latn", "eus_Latn", "fin_Latn", "fra_Latn", "fuv_Latn", "gaz_Latn", "grn_Latn",
    "guj_Gujr", "hat_Latn", "hau_Latn", "heb_Hebr", "hin_Deva", "hin_Latn", "hrv_Latn", "hun_Latn",
    "hye_Armn", "ibo_Latn", "ilo_Latn", "ind_Latn", "isl_Latn", "ita_Latn", "jav_Latn", "jpn_Jpan",
    "kac_Latn", "kan_Knda", "kat_Geor", "kaz_Cyrl", "kea_Latn", "khk_Cyrl", "khm_Khmr", "kin_Latn",
    "kir_Cyrl", "kor_Hang", "lao_Laoo", "lin_Latn", "lit_Latn", "lug_Latn", "luo_Latn", "lvs_Latn",
    "mal_Mlym", "mar_Deva", "mkd_Cyrl", "mlt_Latn", "mri_Latn", "mya_Mymr", "nld_Latn", "nob_Latn",
    "npi_Deva", "npi_Latn", "nso_Latn", "nya_Latn", "ory_Orya", "pan_Guru", "pbt_Arab", "pes_Arab",
    "plt_Latn", "pol_Latn", "por_Latn", "ron_Latn", "rus_Cyrl", "shn_Mymr", "sin_Latn", "sin_Sinh",
    "slk_Latn", "slv_Latn", "sna_Latn", "snd_Arab", "som_Latn", "sot_Latn", "spa_Latn", "srp_Cyrl",
    "ssw_Latn", "sun_Latn", "swe_Latn", "swh_Latn", "tam_Taml", "tel_Telu", "tgk_Cyrl", "tgl_Latn",
    "tha_Thai", "tir_Ethi", "tsn_Latn", "tso_Latn", "tur_Latn", "ukr_Cyrl", "urd_Arab", "urd_Latn",
    "uzn_Latn", "vie_Latn", "war_Latn", "wol_Latn", "xho_Latn", "yor_Latn", "zho_Hans", "zho_Hant",
    "zsm_Latn", "zul_Latn"
]




In [2]:
prompt_belebele= "P: {}\nQ: {}\nA: {}\nB: {}\nC: {}\nD: {}\nPlease choose the correct answer from the options above:"


In [3]:
#this logic is taken from afrobench. 
import re

# Hardcoded choices
choices = ["A", "B", "C", "D"]

# Hardcoded verbalizer mapping
verbalizer = {
    "A": ['a:', 'a', 'a.', '1', '1:', 'a)', '(a)', 'option a', 'option a:', 'option_a:'],
    "B": ['b:', 'b', 'b.', '2', '2:', 'b)', '(b)', 'option b', 'option b:', 'option_b:'],
    "C": ['c:', 'c', 'c.', '3', '3:', 'c)', '(c)', 'option c', 'option c:', 'option_c:'],
    "D": ['d:', 'd', 'd.', '4', '4:', 'd)', '(d)', 'option d', 'option d:', 'option_d:'],
}

map_response = {
    "A": '1',
    "B": '2',
    "C": '3',
    "D": '4',
    "invalid": "invalid"
}

def get_category(prompt):
    completion = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ])
    response = completion.choices[0].message.content
    # print(response)
    choice_patterns = {choice: re.compile(rf"\b{re.escape(choice)}\b", re.IGNORECASE) for choice in choices}

    best_match = None
    best_position = len(response) + 1         
    # Here check for all 4, and choose the best position. 
    for choice, pattern in choice_patterns.items():
        match = pattern.search(response)
        if match and match.start() < best_position:
            best_match = choice
            best_position = match.start()

    # Check against verbalizer if no match found, and take the first hit
    if not best_match and verbalizer:
        for key, synonyms in verbalizer.items():
            for synonym in synonyms:
                # Use \b for word boundaries to avoid matching parts of words
                if re.search(rf"\b{re.escape(synonym)}\b", response.lower(), re.IGNORECASE):
                    best_match = key
                    break
            if best_match:
                break
    # Append result
    if best_match:
        answer = best_match
    else:
        answer = "invalid"

    return response, map_response[answer]

In [7]:
languages_to_run = languages[8:17]

In [8]:
from tqdm import tqdm
result_categories = {}
result_accuracies = {}
gpt_replies = {}

for language_code in languages_to_run:
    completed = []
    if language_code in completed:
        continue
    print(language_code)
    accurate = 0
    
    result_categories[language_code]=[]
    gpt_replies[language_code] = []
    dataset = load_dataset("facebook/belebele", language_code)

    size = len(dataset['test'])
    for i in tqdm(range(size)): #length of devtest
        flores_passage = dataset['test'][i]['flores_passage']
        question = dataset['test'][i]['question']
        mc_answer1 = dataset['test'][i]['mc_answer1']
        mc_answer2 = dataset['test'][i]['mc_answer2']
        mc_answer3 = dataset['test'][i]['mc_answer3']
        mc_answer4 = dataset['test'][i]['mc_answer4']
        
        prompt = prompt_belebele.format(flores_passage, question, mc_answer1, mc_answer2, mc_answer3, mc_answer4)
        reply, category = get_category(prompt)
        gpt_replies[language_code].append(reply)
            
        result_categories[language_code].append(category)
        if category == dataset['test'][i]['correct_answer_num']:
            accurate+=1
    result_accuracies[language_code] = accurate

    df = pd.DataFrame({
    "question": dataset['test']['question'],
    "gpt_answer": result_categories[language_code]
    })

    # Save to CSV
    df.to_csv("belebele_predicted_gpt4o/{}.csv".format(language_code), index=False)

    df2 = pd.DataFrame({
    "question": dataset['test']['question'],
    "gpt_reply": gpt_replies[language_code]
    })

    # Save to CSV
    df2.to_csv("belebele_replies_gpt4o/{}.csv".format(language_code), index=False)
    print(accurate)
print(result_accuracies)

ary_Arab


ary_Arab.jsonl:   0%|          | 0.00/2.84M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/900 [00:00<?, ? examples/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [11:20<00:00,  1.32it/s]


739
arz_Arab


...

324
bul_Cyrl


bul_Cyrl.jsonl:   0%|          | 0.00/3.37M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/900 [00:00<?, ? examples/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [12:05<00:00,  1.24it/s]

846
{'ary_Arab': 739, 'arz_Arab': 787, 'asm_Beng': 776, 'azj_Latn': 781, 'bam_Latn': 324, 'ben_Beng': 795, 'ben_Latn': 710, 'bod_Tibt': 324, 'bul_Cyrl': 846}


In [10]:
done=['hat_Latn', 'afr_Latn', 'snd_Arab', 'nob_Latn', 'fra_Latn', 'rus_Cyrl', 'ron_Latn', 'zho_Hans', 'guj_Gujr', 'ars_Arab', 'kor_Hang', 'hau_Latn', 'mri_Latn', 'swe_Latn', 'ory_Orya', 'war_Latn', 'hye_Armn', 'isl_Latn', 'kat_Geor', 'npi_Latn', 'mlt_Latn', 'slv_Latn', 'arb_Arab', 'tha_Thai', 'urd_Latn', 'sot_Latn', 'bod_Tibt', 'ben_Beng', 'tur_Latn', 'nso_Latn', 'kaz_Cyrl', 'gaz_Latn', 'som_Latn', 'sun_Latn', 'heb_Hebr', 'nld_Latn', 'hrv_Latn', 'lug_Latn', 'mar_Deva', 'tgl_Latn', 'xho_Latn', 'wol_Latn', 'azj_Latn', 'tel_Telu', 'ben_Latn', 'srp_Cyrl', 'pbt_Arab', 'ibo_Latn', 'sna_Latn', 'ell_Grek', 'khk_Cyrl', 'hin_Deva', 'por_Latn', 'luo_Latn', 'mal_Mlym', 'est_Latn', 'lit_Latn', 'tam_Taml', 'tso_Latn', 'yor_Latn', 'ces_Latn', 'kin_Latn', 'sin_Latn', 'ckb_Arab', 'pol_Latn', 'shn_Mymr', 'npi_Deva', 'ukr_Cyrl', 'cat_Latn', 'khm_Khmr', 'hin_Latn', 'amh_Ethi', 'tir_Ethi', 'spa_Latn', 'apc_Arab', 'grn_Latn', 'eng_Latn', 'jpn_Jpan', 'ssw_Latn', 'fin_Latn', 'slk_Latn', 'kea_Latn', 'tgk_Cyrl', 'ita_Latn', 'ilo_Latn', 'als_Latn', 'dan_Latn', 'kir_Cyrl', 'ary_Arab', 'bul_Cyrl', 'arb_Latn', 'nya_Latn', 'tsn_Latn', 'eus_Latn', 'deu_Latn', 'vie_Latn', 'hun_Latn', 'asm_Beng', 'mkd_Cyrl', 'jav_Latn', 'plt_Latn', 'acm_Arab', 'uzn_Latn', 'sin_Sinh', 'ceb_Latn', 'lvs_Latn', 'pan_Guru', 'lao_Laoo', 'fuv_Latn', 'mya_Mymr', 'ind_Latn', 'bam_Latn']
languages = [
    "acm_Arab", "afr_Latn", "als_Latn", "amh_Ethi", "apc_Arab", "arb_Arab", "arb_Latn", "ars_Arab",
    "ary_Arab", "arz_Arab", "asm_Beng", "azj_Latn", "bam_Latn", "ben_Beng", "ben_Latn", "bod_Tibt",
    "bul_Cyrl", "cat_Latn", "ceb_Latn", "ces_Latn", "ckb_Arab", "dan_Latn", "deu_Latn", "ell_Grek",
    "eng_Latn", "est_Latn", "eus_Latn", "fin_Latn", "fra_Latn", "fuv_Latn", "gaz_Latn", "grn_Latn",
    "guj_Gujr", "hat_Latn", "hau_Latn", "heb_Hebr", "hin_Deva", "hin_Latn", "hrv_Latn", "hun_Latn",
    "hye_Armn", "ibo_Latn", "ilo_Latn", "ind_Latn", "isl_Latn", "ita_Latn", "jav_Latn", "jpn_Jpan",
    "kac_Latn", "kan_Knda", "kat_Geor", "kaz_Cyrl", "kea_Latn", "khk_Cyrl", "khm_Khmr", "kin_Latn",
    "kir_Cyrl", "kor_Hang", "lao_Laoo", "lin_Latn", "lit_Latn", "lug_Latn", "luo_Latn", "lvs_Latn",
    "mal_Mlym", "mar_Deva", "mkd_Cyrl", "mlt_Latn", "mri_Latn", "mya_Mymr", "nld_Latn", "nob_Latn",
    "npi_Deva", "npi_Latn", "nso_Latn", "nya_Latn", "ory_Orya", "pan_Guru", "pbt_Arab", "pes_Arab",
    "plt_Latn", "pol_Latn", "por_Latn", "ron_Latn", "rus_Cyrl", "shn_Mymr", "sin_Latn", "sin_Sinh",
    "slk_Latn", "slv_Latn", "sna_Latn", "snd_Arab", "som_Latn", "sot_Latn", "spa_Latn", "srp_Cyrl",
    "ssw_Latn", "sun_Latn", "swe_Latn", "swh_Latn", "tam_Taml", "tel_Telu", "tgk_Cyrl", "tgl_Latn",
    "tha_Thai", "tir_Ethi", "tsn_Latn", "tso_Latn", "tur_Latn", "ukr_Cyrl", "urd_Arab", "urd_Latn",
    "uzn_Latn", "vie_Latn", "war_Latn", "wol_Latn", "xho_Latn", "yor_Latn", "zho_Hans", "zho_Hant",
    "zsm_Latn", "zul_Latn"
]
set(languages)-set(done)

{'arz_Arab',
 'kac_Latn',
 'kan_Knda',
 'lin_Latn',
 'pes_Arab',
 'swh_Latn',
 'urd_Arab',
 'zho_Hant',
 'zsm_Latn',
 'zul_Latn'}